In [1]:
!pip install pennylane
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 61.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
  Attempting uninstall: autograd
    Found existing installation: autograd 1.6.1
    Uninstalling autograd-1.6.1:
      Successfully uninstalled autograd-1.6.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8

In [2]:
import pennylane as qml
import torch
from torch import Tensor
import numpy as np
import torch.nn as nn

from tqdm.notebook import tqdm

from transformers import AutoTokenizer

from torch.utils.data import DataLoader, Dataset
from tensorflow import compat
import tensorflow_datasets as tfds
import scipy

In [3]:
# Fetching pre-trained tokenizer and imdb dataset
tokenizer = AutoTokenizer.from_pretrained("facebook/data2vec-text-base")
vocab_size = len(tokenizer)

# Loading IMDB Dataset from tensorflow datasets
imdb_data = tfds.load("imdb_reviews", shuffle_files=True)
imdb_train_data = tfds.as_dataframe(imdb_data['train']) #.take(64))
imdb_test_data = tfds.as_dataframe(imdb_data['test']) #.take(64))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5JI90U/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5JI90U/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5JI90U/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
Selected_train_data=imdb_train_data.loc[:3999]
Selected_test_data=imdb_test_data.loc[:399]

In [5]:
class D2Tset(Dataset):
  def __init__(self, data_df, tokenizer=None, max_seq_length=None, padding=True, transform=None, target_transform=None):
    self.df = data_df
    self.tokenizer = tokenizer
    self.max_seq_length = max_seq_length
    self.transform = transform
    self.target_transform = target_transform
    self.pad = padding

    if self.max_seq_length is not None:
      self.truncation = True
    else:
      self.truncation = False

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, idx):
    data = self.df.iloc[idx]
    label = data['label']
    feature = self._txt2vec(data['text'])
    return feature.input_ids.squeeze(0), label

  def _txt2vec(self, txt):
    if isinstance(txt, bytes):
        txt = compat.as_str_any(txt)
    tokenized = self.tokenizer(txt, return_tensors='pt', truncation=self.truncation, padding=self.pad, max_length=self.max_seq_length)
    return tokenized

In [6]:
# Using dataloader for the imdb data
imdb_trainset = D2Tset(data_df=Selected_train_data, tokenizer=tokenizer, max_seq_length=128, padding='max_length', transform=None, target_transform=None)
imdb_trainloader = DataLoader(imdb_trainset, shuffle=True, batch_size=400)

imdb_testset = D2Tset(data_df=Selected_test_data, tokenizer=tokenizer, max_seq_length=128, padding='max_length', transform=None, target_transform=None)
imdb_testloader = DataLoader(imdb_testset, shuffle=False, batch_size=400)
imdb_feature, imdb_label = next(iter(imdb_trainloader))
print(imdb_feature.shape, imdb_label.shape)

torch.Size([400, 128]) torch.Size([400])


In [7]:
import functools
import inspect
import math
from collections.abc import Iterable
from typing import Callable, Dict, Union, Any

from pennylane.qnode import QNode

try:
    import torch
    from torch.nn import Module

    TORCH_IMPORTED = True
except ImportError:
    # The following allows this module to be imported even if PyTorch is not installed. Users
    # will instead see an ImportError when instantiating the TorchLayer.
    from unittest.mock import Mock

    Module = Mock
    TORCH_IMPORTED = False


class TorchLayer(Module):
    def __init__(self,qnode,weights):
        if not TORCH_IMPORTED:
            raise ImportError(
                "TorchLayer requires PyTorch. PyTorch can be installed using:\n"
                "pip install torch\nAlternatively, "
                "visit https://pytorch.org/get-started/locally/ for detailed "
                "instructions."
            )
        super().__init__()

        #weight_shapes = {
        #    weight: (tuple(size) if isinstance(size, Iterable) else () if size == 1 else (size,))
        #    for weight, size in weight_shapes.items()
        #}

        # validate the QNode signature, and convert to a Torch QNode.
        # TODO: update the docstring regarding changes to restrictions when tape mode is default.
        #self._signature_validation(qnode, weight_shapes)
        self.qnode = qnode
        self.qnode.interface = "torch"

        self.qnode_weights = weights

    def forward(self, inputs):  # pylint: disable=arguments-differ
        """Evaluates a forward pass through the QNode based upon input data and the initialized
        weights.

        Args:
            inputs (tensor): data to be processed

        Returns:
            tensor: output data
        """

        if len(inputs.shape) > 1:
            # If the input size is not 1-dimensional, unstack the input along its first dimension,
            # recursively call the forward pass on each of the yielded tensors, and then stack the
            # outputs back into the correct shape
            reconstructor = [self.forward(x) for x in torch.unbind(inputs)]
            return torch.stack(reconstructor)

        # If the input is 1-dimensional, calculate the forward pass as usual
        return self._evaluate_qnode(inputs)


    def _evaluate_qnode(self, x):
        """Evaluates the QNode for a single input datapoint.

        Args:
            x (tensor): the datapoint

        Returns:
            tensor: output datapoint
        """
        kwargs = {
            **{self.input_arg: x},
            **{arg: weight.to(x) for arg, weight in self.qnode_weights.items()},
        }
        res = self.qnode(**kwargs)

        if isinstance(res, torch.Tensor):
            return res.type(x.dtype)

        return torch.hstack(res).type(x.dtype)

    def __str__(self):
        detail = "<Quantum Torch Layer: func={}>"
        return detail.format(self.qnode.func.__name__)

    __repr__ = __str__
    _input_arg = "inputs"

    @property
    def input_arg(self):
        """Name of the argument to be used as the input to the Torch layer. Set to ``"inputs"``."""
        return self._input_arg

In [8]:
class qrnn(torch.nn.Module):

    def __init__(self, anc_q, n_qubs, seq_num, D):
        super().__init__()
        self.num_anc_q=anc_q
        self.seq_num=seq_num
        self.n_qubs=n_qubs
        self.num_ansatz_q=anc_q+n_qubs
        self.n_input_each_blc=2**n_qubs#(Denc+2)
        self.D=D
        #self.Denc=Denc
        self.num_q=n_qubs*self.seq_num+anc_q

        self.init_params=torch.nn.Parameter((np.pi/4) * (2 * torch.randn(self.num_ansatz_q*(self.D+2)*self.seq_num) - 1))

        self.dev = qml.device("default.qubit", wires=self.num_q)
        self.qnod=qml.QNode(self.circuit, self.dev, interface="torch")
        self.weight = {"weights": self.init_params}
        self.linear = TorchLayer(self.qnod, self.weight)

    def circuit(self,inputs,weights):
        self.num_para_per_bloc=self.num_ansatz_q*(self.D+2)
        index=0
        #inputs=inputs*np.pi/25000
        #for i in range(self.seq_num):
        #    in_s=inputs[i*self.n_input_each_blc:i*self.n_input_each_blc+self.n_input_each_blc]
        #    qml.AmplitudeEmbedding(in_s,wires=range(self.num_anc_q+i*self.n_input_each_blc,self.num_ansatz_q+i*self.n_input_each_blc), normalize=True)
        self.encoding(inputs)
        for i in range(self.seq_num):
            #self.encoding(inputs[i*self.n_param_each_blc:i*self.n_param_each_blc+self.n_param_each_blc])
            self.ansatz(weights[i*self.num_para_per_bloc:i*self.num_para_per_bloc+self.num_para_per_bloc])
            index+=self.num_ansatz_q*(3*self.D+2)*self.seq_num
            if i!=self.seq_num-1:
                for j in range(self.n_qubs):
                    q1=j+self.num_anc_q
                    q2=(i+1)*self.n_qubs+j+self.num_anc_q
                    qml.SWAP(wires=[q1,q2])
        return qml.expval(qml.PauliZ(0))

    def ansatz(self,weights):
        indx=0
        for j in range(self.num_ansatz_q):
            qml.RY(weights[indx+1],wires=j)
            qml.RX(weights[indx],wires=j)
            indx+=2
        for i in range(self.D):
            for j in range(self.num_ansatz_q):
                #qml.IsingZZ(weights[indx],wires=[j,(j+1)%self.num_q])
                qml.CNOT(wires=[j,(j+1)%self.num_q])
                #qml.RZ(weights[indx],wires=(j+1)%self.num_q)
                #qml.CNOT(wires=[j,(j+1)%self.num_q])
                #indx+=1
            for j in range(self.num_ansatz_q):
                #qml.RX(weights[indx],wires=j)
                qml.RY(weights[indx],wires=j)
                indx+=1

    def encoding(self, input):
        bit_capacity=2**self.n_qubs
        #cf=np.zeros(bit_capacity)
        ind=0
        #for i in range(bit_capacity):
        cf=input[ind:ind+bit_capacity]
        ind+=bit_capacity
        norm_const=scipy.linalg.norm(cf)
        if norm_const==0:
            cf=np.ones(bit_capacity)/np.sqrt(bit_capacity)
        else:
            stat_vec=cf/norm_const
        for j in range(1,self.seq_num):
            #cf=np.zeros(bit_capacity)
            cf=input[ind:ind+bit_capacity]
            ind+=bit_capacity
            #for i in range(bit_capacity):
                #cf[i]=input[ind]
                #ind+=1
            norm_const=scipy.linalg.norm(cf)
            if norm_const==0:
                cf=np.ones(bit_capacity)/np.sqrt(bit_capacity)
            else:
                cf=cf/norm_const
            stat_vec=np.kron(stat_vec,cf)
        qml.QubitStateVector(stat_vec, wires=range(self.num_anc_q,self.num_q))


    def forward(self, x):

        x=self.linear(x)
        return torch.sigmoid(x)

In [9]:
def model_trainer(model, n_epochs, trainloader):
  '''
  Model trainer to train QRNN
  Parameters:
    model (PyTorch Model): QRNN model for text or image classification with correct sizes specified
    n_epochs (int): Number of epochs to train for.
    trainloader (PyTorch Dataloader): Dataloader containing the dataset.
  '''
  train_loss = []
  optimizer = torch.optim.Adam(lr=0.03, params=model.parameters())
  criterion = torch.nn.CrossEntropyLoss() #nn.BCELoss()
  pbar = tqdm(total=len(trainloader), leave=True)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.train()
  model.to(device)
  for epoch in range(n_epochs):
    for batch, (feature, label) in enumerate(trainloader):
      feature, label = feature.to(device), label.to(device)
      print(feature)
      optimizer.zero_grad()
      predictions = model(feature.squeeze())
      loss = criterion(predictions.squeeze(), label.float())
      print(loss)
      loss.backward()
      optimizer.step()
      pbar.update()
      pbar.desc = f"Epoch: {epoch} | Batch: {batch} | Loss {loss}"
      train_loss.append(loss.cpu().detach().numpy())
      #print(acc)
      #print(loss)
    pbar.refresh()
  pbar.close()
  return model, train_loss


def model_tester(model, testloader):
  '''
  Model test to train QRNN
  Parameters:
    model (PyTorch Model): QRNN model for text or image classification with correct sizes specified
    testloader (PyTorch Dataloader): Dataloader containing the test dataset.
  '''
  preds = []
  labels = []
  pbar = tqdm(total=len(testloader), leave=True)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.eval()
  model.to(device)
  for batch, (feature, label) in enumerate(testloader):
    feature, label = feature.to(device), label.to(device)
    with torch.no_grad():
      predictions = model(feature.squeeze())
      preds.append(predictions.cpu().numpy())
      labels.append(label.cpu().numpy())
    pbar.update()
    pbar.desc = f"Batch: {batch}"
  pbar.refresh()
  pbar.close()
  preds = np.array([1 if pred>=0.5 else 0 for pred in preds])
  labels = np.array(labels)
  acc = (preds == labels).sum() / len(preds)
  return preds, acc, labels

In [10]:
model=qrnn(1,6,2,1)

In [ ]:
train_loss = []
optimizer = torch.optim.Adam(lr=0.03, params=model.parameters())
criterion = nn.BCELoss()#torch.nn.CrossEntropyLoss() #nn.BCELoss()
pbar = tqdm(total=len(imdb_trainloader), leave=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.train()
model.to(device)
n_epochs=20
for epoch in range(n_epochs):
  for batch, (feature, label) in enumerate(imdb_trainloader):
    feature, label = feature.to(device), label.to(device)
    feature=feature/50000
    #print(feature)
    optimizer.zero_grad()
    predictions = model(feature.squeeze())
    loss = criterion(predictions.squeeze(), label.float())
    print(loss)
    loss.backward()
    optimizer.step()
    pbar.update()
    pbar.desc = f"Epoch: {epoch} | Batch: {batch} | Loss {loss}"
    train_loss.append(loss.cpu().detach().numpy())


  0%|          | 0/10 [00:00<?, ?it/s]

tensor(0.7013, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6978, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6964, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6959, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6947, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6935, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6932, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6935, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6937, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6908, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6936, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6921, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6919, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6948, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6918, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6932, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6922, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6946, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.6943, grad_fn=<Bina

In [ ]:
acc=0
pbar = tqdm(total=len(imdb_trainloader), leave=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)
for batch, (feature, label) in enumerate(imdb_trainloader):
  preds = []
  labels = []
  feature, label = feature.to(device), label.to(device)
  with torch.no_grad():
    feature=feature/50000
    predictions = model(feature.squeeze())
    preds.append(predictions.cpu().numpy())
    labels.append(label.cpu().numpy())
  preds = np.array([1 if pred >=0.5 else 0 for pred in preds[0]])
  labels = np.array(labels)
  acc+= (preds == labels).sum() / len(preds)

  pbar.update()
  pbar.desc = f"Batch: {batch}"
pbar.refresh()
pbar.close()
acc/10

In [ ]:
preds = []
labels = []
pbar = tqdm(total=len(imdb_testloader), leave=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device)
for batch, (feature, label) in enumerate(imdb_testloader):
  feature, label = feature.to(device), label.to(device)
  with torch.no_grad():
    feature=feature/50000
    predictions = model(feature.squeeze())
    preds.append(predictions.cpu().numpy())
    labels.append(label.cpu().numpy())
  pbar.update()
  pbar.desc = f"Batch: {batch}"
pbar.refresh()
pbar.close()
preds = np.array([1 if pred >=0.5 else 0 for pred in preds[0]])
labels = np.array(labels)
acc = (preds == labels).sum() / len(preds)


In [ ]:
acc

0.5075

In [ ]:
preds

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
optimizer = torch.optim.Adam(lr=0.005, params=model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

100


In [ ]:
for iepoch in tqdm(range(50)):

    optimizer.zero_grad()

    idx = torch.randperm(2000)[:400]
    X_batch = new_X_train[idx]
    predictions=model(X_batch.float()).squeeze(1)

    label_batch=label[idx]
    loss = criterion(predictions, label_batch.float())
    acc = binary_accuracy(predictions,label_batch)
    print('')
    print('Accuracy:',acc)
    print('')
    print(loss)
    loss.backward()
    optimizer.step()